<font size="5">Imports</font>

In [192]:
import pandas as pd
import numpy as np



<font size="5">Bring in table NHL Stanley Cup Champions.</font>
<br>
<font size="5">Source: Table was scraped from NHL website.</font>

In [193]:
df_champs = pd.read_csv('NHL_Champions.txt')
df_champs.head(5)

,Year,Team,Coach
0,2022,Colorado Avalanche,Coach Jared Bednar
1,2021,Tampa Bay Lightning,Coach Jon Cooper
2,2020,Tampa Bay Lightning,Coach Jon Cooper
3,2019,St. Louis Blues,Coach Craig Berube
4,2018,Washington Capitals,Coach Barry Trotz


<font size="5">Championship teams and the years they've won the Stanley Cup.</font>

In [194]:
df_teams_years_won = df_champs[['Team', 'Year']].copy()
df_teams_years_won = df_teams_years_won.groupby(by=['Team', 'Year']).count() # Empty Dataframe, only MultiIndex remains.

# Argument sort_remaining works left to right. Therefore, need to sort in order of right to left.
df_teams_years_won.sort_index(level=1, ascending=False, inplace=True, sort_remaining=False) # Sort the years, descending.
df_teams_years_won.sort_index(level=0, ascending=True, inplace=True, sort_remaining=False) # Sort the team names, ascending.

# .head(20) to show the new 2022 Champs, the Colorado Avalanche!
df_teams_years_won.head(20)

Empty DataFrame
Columns: []
Index: [(Anaheim Ducks, 2007), (Boston Bruins, 2011), (Boston Bruins, 1972), (Boston Bruins, 1970), (Boston Bruins, 1941), (Boston Bruins, 1939), (Boston Bruins, 1929), (Calgary Flames, 1989), (Carolina Hurricanes, 2006), (Chicago Black Hawks, 1961), (Chicago Black Hawks, 1938), (Chicago Black Hawks, 1934), (Chicago Blackhawks, 2015), (Chicago Blackhawks, 2013), (Chicago Blackhawks, 2010), (Colorado Avalanche, 2022), (Colorado Avalanche, 2001), (Colorado Avalanche, 1996), (Dallas Stars, 1999), (Detroit Red Wings, 2008)]

In [195]:
Championship_Years_filename = 'Championship_Teams_Years.xlsx'
with pd.ExcelWriter(Championship_Years_filename) as writer:
    df_teams_years_won.to_excel(writer, sheet_name='Champions Years')

<font size="5">Championship teams and the years they've won the Stanley Cup.</font>
<br>
<font size="4">Includes subtotal row for count of years won.</font>

In [196]:
df_wins_subtotals = df_champs[['Team', 'Year']].copy()
df_wins_subtotals = df_wins_subtotals.groupby(by=['Team', 'Year']).count() # Empty DataFrame, only multi-index remains.
# Multi-index sort by team asc, year desc.
df_wins_subtotals.sort_index(level=1, ascending=False, inplace=True, sort_remaining=False)
df_wins_subtotals.sort_index(level=0, ascending=True, inplace=True, sort_remaining=False)
# Get a unique list of the Championship teams.
teams_won_list = sorted(list(set([team[0] for team in df_wins_subtotals.index.to_list()])),
                         reverse=False) # Stay consistent to the index sorts.

# Create a Count_Year column from the second level of the multi-index.
df_wins_subtotals['Count_Year'] = [y[1] for y in df_wins_subtotals.index.to_list()]
concat_list = []
for team in teams_won_list:
    # Filter to a Championship team.
    team_rows = df_wins_subtotals.query(f'Team=="{team}"')    
    # print(team_rows)
    # Create an np.nan row.
    subtotal_row = pd.Series([np.nan], index=[f'{team} Total Stanley Cups'], name='Count_Year').to_frame()
    # print(subtotal_row)
    # Fill the np.nan with the selected team's count of years won.
    subtotal_row['Count_Year'].fillna(team_rows['Count_Year'].count(), inplace=True)
    # Concat the filtered team with the subtotal row.
    df_team_subtotal = pd.concat([team_rows, subtotal_row])  
    # Use concat_list to store the filtered team with subtotal row.
    concat_list.append(df_team_subtotal)

# Put it all together.
df_wins_subtotals_2 = pd.concat(concat_list)

# Clean up the results.
df_wins_subtotals_2['Count_Year'] = pd.to_numeric(df_wins_subtotals_2['Count_Year']).astype('int64')

df_wins_subtotals_2['Teams'] = [t[0] if (len(t[0]) > 1) else t for t in df_wins_subtotals_2.index.to_list()]

df_wins_subtotals_2 = df_wins_subtotals_2[['Teams', 'Count_Year']]
df_wins_subtotals_2.reset_index(inplace=True, drop=True)
df_wins_subtotals_2.rename(columns={'Count_Year': 'Years_Won'}, inplace=True)

# .head(25) to see the new Stanley Cup champs and their subtotal of years won!
df_wins_subtotals_2.head(25)


,Teams,Years_Won
0,Anaheim Ducks,2007
1,Anaheim Ducks Total Stanley Cups,1
2,Boston Bruins,2011
3,Boston Bruins,1972
4,Boston Bruins,1970
5,Boston Bruins,1941
6,Boston Bruins,1939
7,Boston Bruins,1929
8,Boston Bruins Total Stanley Cups,6
9,Calgary Flames,1989


In [197]:
# Send the results to excel for end users.
teams_by_years_won_filename = 'teams_by_years_won.xlsx'
with pd.ExcelWriter(teams_by_years_won_filename) as writer:
    df_wins_subtotals_2.to_excel(writer, sheet_name='Teams_Wins', index=False)

<font size="5">Which Team has the most Stanley Cup wins?</font>

In [198]:
df_champs_wins = df_champs[['Team', 'Year']].copy()
df_champs_wins.rename(columns={'Year':'Stanley_Cup_Wins'}, inplace=True)
df_champs_wins = df_champs_wins.groupby(by=['Team'], as_index=False).count()
df_champs_wins.sort_values(by=['Stanley_Cup_Wins'], ascending=False, inplace=True)
df_champs_wins.reset_index(drop=True, inplace=True)
df_champs_wins.head(10)

,Team,Stanley_Cup_Wins
0,Montreal Canadiens,22
1,Toronto Maple Leafs,11
2,Detroit Red Wings,11
3,Boston Bruins,6
4,Edmonton Oilers,5
5,Pittsburgh Penguins,5
6,New York Islanders,4
7,New York Rangers,4
8,Tampa Bay Lightning,3
9,Chicago Black Hawks,3


<font size="5">Of the current 2022 NHL teams, which team(s) have yet to win a Stanley Cup?</font>

In [199]:
# Updated: Scraped NHL website for a list of current 2022 NHL teams.
df_current_teams = pd.read_csv('NHL_Teams.csv',
                              usecols=['NHL_Teams'])
print(df_current_teams.shape) # 32 teams, as expected.
df_current_teams.head(5)

(32, 1)


,NHL_Teams
0,Carolina Hurricanes
1,Columbus Blue Jackets
2,New Jersey Devils
3,New York Islanders
4,New York Rangers
